In [1]:
import pylab
import calendar
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime
from sklearn.preprocessing  import StandardScaler, OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
raw_data = pd.read_csv("input/train.csv")
test_data = pd.read_csv("input/test.csv")

check data

In [3]:
raw_data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


break datetime to date, hour, weekday, month

In [4]:
def break_datetime(data):
    data['date'] = data.datetime.apply(lambda x: x.split()[0])
    data['hour'] = data.datetime.apply(lambda x: x.split()[1].split(':')[0]).astype('int')
    data['weekday'] = data.date.apply(lambda dateString: datetime.strptime(dateString, '%Y-%m-%d').weekday())
    data['month'] = data.date.apply(lambda dateString: datetime.strptime(dateString, '%Y-%m-%d').month)
    data = data.drop(['datetime', 'date'], axis=1)
    return data

In [5]:
data_no_datetime = break_datetime(raw_data)

In [6]:
data_no_datetime

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,weekday,month
0,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,0,5,1
1,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,1,5,1
2,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,2,5,1
3,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,3,5,1
4,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,19,2,12
10882,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,20,2,12
10883,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,21,2,12
10884,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,22,2,12


change weather and season to one-hot columns

In [7]:
def one_hot_transform(data):
    season_one_hot = OneHotEncoder()
    scol = season_one_hot.fit_transform(data['season'].values.reshape(-1, 1)).toarray()
    scol = pd.DataFrame(scol)
    data = pd.concat([data, scol], axis=1)
    data.drop(['season'], axis=1)

    weather_one_hot = OneHotEncoder()
    wcol = weather_one_hot.fit_transform(data['weather'].values.reshape(-1, 1)).toarray()
    wcol = pd.DataFrame(wcol)
    data = pd.concat([data, wcol], axis=1)
    data.drop(['weather'], axis=1)
    
    return data

In [8]:
data_one_hot = one_hot_transform(data_no_datetime)

In [9]:
data_one_hot

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,...,weekday,month,0,1,2,3,0,1,2,3
0,1,0,0,1,9.84,14.395,81,0.0000,3,13,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,0,0,1,9.02,13.635,80,0.0000,8,32,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,0,0,1,9.02,13.635,80,0.0000,5,27,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,0,0,1,9.84,14.395,75,0.0000,3,10,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,0,0,1,9.84,14.395,75,0.0000,0,1,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,7,329,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10882,4,0,1,1,14.76,17.425,57,15.0013,10,231,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10883,4,0,1,1,13.94,15.910,61,15.0013,4,164,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10884,4,0,1,1,13.94,17.425,61,6.0032,12,117,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


shuffle and split

In [10]:
def split_data(data):
    data = data.sample(frac=1) #shuffle
    test_index = int(data.shape[0] * 0.8)
    y_training = data['count'][:test_index]
    y_validation = data['count'][test_index:]
    data = data.drop(['casual', 'registered', 'count', 'atemp', 'season'], axis=1)
    return data[:test_index], y_training, data[test_index:], y_validation

In [11]:
x_training, y_training, x_validation, y_validation = split_data(data_one_hot)

In [27]:
x_training

,holiday,workingday,weather,temp,humidity,windspeed,hour,weekday,month,0,1,2,3,0,1,2,3
8793,0,1,2,31.98,55,11.0014,18,2,8,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
5101,0,1,3,20.50,100,12.9980,15,1,12,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1161,0,0,1,14.76,66,8.9981,1,6,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6141,0,0,2,4.10,46,46.0022,3,6,2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1639,0,1,1,13.94,76,6.0032,5,3,4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6542,0,1,1,15.58,32,23.9994,20,4,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2625,0,1,1,24.60,83,0.0000,7,4,6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
6408,0,0,2,12.30,52,19.9995,6,6,3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2174,0,1,3,22.96,83,27.9993,12,1,5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [28]:
y_training

8793    862
5101     45
1161     33
6141     14
1639     14
       ... 
6542    159
2625    250
6408      5
2174    153
989     204
Name: count, Length: 8708, dtype: int64

In [14]:
def build_mode(layer_units, input_size):
    layers = []
    layers.append(tf.keras.layers.Dense(units=layer_units[0], activation='relu', input_shape=(input_size,)))
    for i in range(1,len(layer_units)):
        layers.append(tf.keras.layers.Dense(units=layer_units[i], activation='relu'))
    layers.append(tf.keras.layers.Dense(units=1, activation='linear'))
    return tf.keras.models.Sequential(layers=layers)

In [15]:
model = build_mode([17], 17)

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='mse')
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 17)                306       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 18        
Total params: 324
Trainable params: 324
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(x_training, y_training, epochs=100, batch_size=128, validation_data=(x_validation, y_validation))

Train on 8708 samples, validate on 2178 samples
Epoch 1/100
8708/8708 [==============================] - 0s 32us/sample - loss: 68441.4909 - val_loss: 65509.3667
Epoch 2/100
8708/8708 [==============================] - 0s 10us/sample - loss: 60929.0580 - val_loss: 58563.1868
Epoch 3/100
8708/8708 [==============================] - 0s 10us/sample - loss: 53585.0946 - val_loss: 50528.8667
Epoch 4/100
8708/8708 [==============================] - 0s 9us/sample - loss: 44053.2044 - val_loss: 39381.7172
Epoch 5/100
8708/8708 [==============================] - 0s 11us/sample - loss: 35434.1513 - val_loss: 33832.8758
Epoch 6/100
8708/8708 [==============================] - 0s 10us/sample - loss: 32050.3809 - val_loss: 31783.8660
Epoch 7/100
8708/8708 [==============================] - 0s 11us/sample - loss: 30511.0018 - val_loss: 30373.2029
Epoch 8/100
8708/8708 [==============================] - 0s 11us/sample - loss: 29117.4544 - val_loss: 29067.3888
Epoch 9/100
8708/8708 [==================

8708/8708 [==============================] - 0s 11us/sample - loss: 19163.1264 - val_loss: 19850.7286
Epoch 73/100
8708/8708 [==============================] - 0s 11us/sample - loss: 19087.2988 - val_loss: 19777.5284
Epoch 74/100
8708/8708 [==============================] - 0s 10us/sample - loss: 19019.1237 - val_loss: 19733.6079
Epoch 75/100
8708/8708 [==============================] - 0s 11us/sample - loss: 18946.6480 - val_loss: 19632.6844
Epoch 76/100
8708/8708 [==============================] - 0s 10us/sample - loss: 18891.1842 - val_loss: 19546.6592
Epoch 77/100
8708/8708 [==============================] - 0s 11us/sample - loss: 18825.7528 - val_loss: 19476.2799
Epoch 78/100
8708/8708 [==============================] - 0s 11us/sample - loss: 18754.7319 - val_loss: 19431.5420
Epoch 79/100
8708/8708 [==============================] - 0s 9us/sample - loss: 18683.2699 - val_loss: 19320.8506
Epoch 80/100
8708/8708 [==============================] - 0s 8us/sample - loss: 18619.7415 - v

loss ~ 17561

increase lr

In [17]:
model_lr = build_mode([17], 17)

model_lr.compile(optimizer=tf.keras.optimizers.Adam(lr=0.008), loss='mse')
model_lr.summary()
history = model_lr.fit(x_training, y_training, epochs=100, batch_size=128, validation_data=(x_validation, y_validation))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 17)                306       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 18        
Total params: 324
Trainable params: 324
Non-trainable params: 0
_________________________________________________________________
Train on 8708 samples, validate on 2178 samples
Epoch 1/100
8708/8708 [==============================] - 0s 25us/sample - loss: 39912.3266 - val_loss: 30681.6908
Epoch 2/100
8708/8708 [==============================] - 0s 9us/sample - loss: 25983.1805 - val_loss: 23434.6897
Epoch 3/100
8708/8708 [==============================] - 0s 9us/sample - loss: 21856.6594 - val_loss: 22147.0015
Epoch 4/100
8708/8708 [==============================] - 0s 9us/sample - loss: 21053.0463 - val_loss: 21576.1528
Epoch 5/100
8708/

Epoch 67/100
8708/8708 [==============================] - 0s 8us/sample - loss: 15288.3410 - val_loss: 16083.5060
Epoch 68/100
8708/8708 [==============================] - 0s 8us/sample - loss: 15316.0116 - val_loss: 16031.2425
Epoch 69/100
8708/8708 [==============================] - 0s 8us/sample - loss: 15269.6458 - val_loss: 15986.8060
Epoch 70/100
8708/8708 [==============================] - 0s 7us/sample - loss: 15312.5386 - val_loss: 16041.2965
Epoch 71/100
8708/8708 [==============================] - 0s 8us/sample - loss: 15265.5598 - val_loss: 16127.2409
Epoch 72/100
8708/8708 [==============================] - 0s 9us/sample - loss: 15270.7281 - val_loss: 16298.2265
Epoch 73/100
8708/8708 [==============================] - 0s 9us/sample - loss: 15324.0117 - val_loss: 16198.0933
Epoch 74/100
8708/8708 [==============================] - 0s 10us/sample - loss: 15328.1433 - val_loss: 16019.6370
Epoch 75/100
8708/8708 [==============================] - 0s 8us/sample - loss: 15293.6

loss ~ 15161

try scale data

In [18]:
def scale_data(data, validation, column):
    ss = StandardScaler()
    d = data[column].values.reshape(-1, 1)
    scaler = ss.fit(d)
    scaled = ss.transform(d)
    scaled_data = data.copy()
    scaled_data[column] = scaled
    
    val_scale = scaler.transform(validation[column].values.reshape(-1,1))
    scaled_val = validation.copy()
    scaled_val[column] = val_scale
    
    return scaled_data, scaled_val, scaler

In [19]:
data_one_hot

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,...,weekday,month,0,1,2,3,0,1,2,3
0,1,0,0,1,9.84,14.395,81,0.0000,3,13,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,0,0,1,9.02,13.635,80,0.0000,8,32,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,0,0,1,9.02,13.635,80,0.0000,5,27,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,0,0,1,9.84,14.395,75,0.0000,3,10,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,0,0,1,9.84,14.395,75,0.0000,0,1,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,7,329,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10882,4,0,1,1,14.76,17.425,57,15.0013,10,231,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10883,4,0,1,1,13.94,15.910,61,15.0013,4,164,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10884,4,0,1,1,13.94,17.425,61,6.0032,12,117,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [21]:
scaled_temp, x_val_temp, temp_scaler = scale_data(x_training, x_validation, 'temp')

scaled_temp

,holiday,workingday,weather,temp,humidity,windspeed,hour,weekday,month,0,1,2,3,0,1,2,3
8793,0,1,2,1.503418,55,11.0014,18,2,8,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
5101,0,1,3,0.034438,100,12.9980,15,1,12,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1161,0,0,1,-0.700053,66,8.9981,1,6,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6141,0,0,2,-2.064106,46,46.0022,3,6,2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1639,0,1,1,-0.804980,76,6.0032,5,3,4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6542,0,1,1,-0.595126,32,23.9994,20,4,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2625,0,1,1,0.559073,83,0.0000,7,4,6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
6408,0,0,2,-1.014834,52,19.9995,6,6,3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2174,0,1,3,0.349219,83,27.9993,12,1,5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
scaled_temp_hum, x_val_temp_hum, humidity_scaler = scale_data(scaled_temp, x_val_temp, 'humidity')

scaled_temp_hum

,holiday,workingday,weather,temp,humidity,windspeed,hour,weekday,month,0,1,2,3,0,1,2,3
8793,0,1,2,1.503418,-0.361294,11.0014,18,2,8,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
5101,0,1,3,0.034438,1.974375,12.9980,15,1,12,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1161,0,0,1,-0.700053,0.209648,8.9981,1,6,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6141,0,0,2,-2.064106,-0.828428,46.0022,3,6,2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1639,0,1,1,-0.804980,0.728685,6.0032,5,3,4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6542,0,1,1,-0.595126,-1.555080,23.9994,20,4,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2625,0,1,1,0.559073,1.092012,0.0000,7,4,6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
6408,0,0,2,-1.014834,-0.517005,19.9995,6,6,3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2174,0,1,3,0.349219,1.092012,27.9993,12,1,5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
scaled_temp_hum_wind, x_val_temp_hum_wind, wind_scaler = scale_data(scaled_temp_hum, x_val_temp_hum, 'windspeed')

scaled_temp_hum_wind

,holiday,workingday,weather,temp,humidity,windspeed,hour,weekday,month,0,1,2,3,0,1,2,3
8793,0,1,2,1.503418,-0.361294,-0.219506,18,2,8,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
5101,0,1,3,0.034438,1.974375,0.024304,15,1,12,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1161,0,0,1,-0.700053,0.209648,-0.464135,1,6,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6141,0,0,2,-2.064106,-0.828428,4.054545,3,6,2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1639,0,1,1,-0.804980,0.728685,-0.829851,5,3,4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6542,0,1,1,-0.595126,-1.555080,1.367718,20,4,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2625,0,1,1,0.559073,1.092012,-1.562920,7,4,6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
6408,0,0,2,-1.014834,-0.517005,0.879278,6,6,3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2174,0,1,3,0.349219,1.092012,1.856158,12,1,5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [26]:
x_val_temp_hum_wind

,holiday,workingday,weather,temp,humidity,windspeed,hour,weekday,month,0,1,2,3,0,1,2,3
3228,0,1,1,1.188636,0.469166,0.879278,10,3,8,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
9483,0,1,3,0.978782,1.092012,2.100786,12,1,9,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7578,0,1,3,0.454146,1.092012,-0.464135,3,1,5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5827,0,1,2,-0.175417,0.780589,1.123907,0,2,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3388,0,1,1,0.873855,-0.413198,-0.464135,2,3,8,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971,0,1,1,-0.280344,1.351530,-1.562920,1,0,5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7160,1,0,1,1.608345,-1.503177,1.612347,17,0,4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
8027,0,1,1,0.873855,0.157744,0.512744,20,3,6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7674,0,0,1,0.034438,-0.724620,-0.707946,3,5,5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [25]:
model_lr = build_mode([17], 17)

model_lr.compile(optimizer=tf.keras.optimizers.Adam(lr=0.008), loss='mse')
model_lr.summary()
history = model_lr.fit(scaled_temp_hum_wind, y_training, epochs=100, batch_size=128, validation_data=(x_val_temp_hum_wind, y_validation))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 17)                306       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 18        
Total params: 324
Trainable params: 324
Non-trainable params: 0
_________________________________________________________________
Train on 8708 samples, validate on 2178 samples
Epoch 1/100
8708/8708 [==============================] - 0s 27us/sample - loss: 49073.1386 - val_loss: 28493.8928
Epoch 2/100
8708/8708 [==============================] - 0s 10us/sample - loss: 25302.2119 - val_loss: 24753.9509
Epoch 3/100
8708/8708 [==============================] - 0s 9us/sample - loss: 23358.8703 - val_loss: 23559.6425
Epoch 4/100
8708/8708 [==============================] - 0s 9us/sample - loss: 22267.5057 - val_loss: 22533.3330
Epoch 5/100
8708

8708/8708 [==============================] - 0s 10us/sample - loss: 14923.1027 - val_loss: 15713.4079
Epoch 67/100
8708/8708 [==============================] - 0s 10us/sample - loss: 14885.9114 - val_loss: 15757.4051
Epoch 68/100
8708/8708 [==============================] - 0s 9us/sample - loss: 14854.5813 - val_loss: 15688.6465
Epoch 69/100
8708/8708 [==============================] - 0s 9us/sample - loss: 14839.1351 - val_loss: 15616.0771
Epoch 70/100
8708/8708 [==============================] - 0s 9us/sample - loss: 14952.6601 - val_loss: 15625.8377
Epoch 71/100
8708/8708 [==============================] - 0s 8us/sample - loss: 14798.7466 - val_loss: 15561.3038
Epoch 72/100
8708/8708 [==============================] - 0s 8us/sample - loss: 14760.7583 - val_loss: 15602.9567
Epoch 73/100
8708/8708 [==============================] - 0s 8us/sample - loss: 14733.8936 - val_loss: 15542.6035
Epoch 74/100
8708/8708 [==============================] - 0s 9us/sample - loss: 14738.8624 - val_lo

loss ~ 14538   better, not much, abondone scale

complex models

In [29]:
model_cplx = build_mode([128, 64], 17) # 1 hidden layer

model_cplx.compile(optimizer=tf.keras.optimizers.Adam(lr=0.008), loss='mse')
model_cplx.summary()
history = model_cplx.fit(x_training, y_training, epochs=100, batch_size=128, validation_data=(x_validation, y_validation))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 128)               2304      
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 10,625
Trainable params: 10,625
Non-trainable params: 0
_________________________________________________________________
Train on 8708 samples, validate on 2178 samples
Epoch 1/100
8708/8708 [==============================] - 0s 28us/sample - loss: 26919.3496 - val_loss: 21357.2058
Epoch 2/100
8708/8708 [==============================] - 0s 10us/sample - loss: 19712.4195 - val_loss: 18973.6633
Epoch 3/100
8708/8708 [==============================] - 0s 11us/sample - loss: 18291.0849 - val_loss: 2

Epoch 66/100
8708/8708 [==============================] - 0s 10us/sample - loss: 7810.5621 - val_loss: 7236.2408
Epoch 67/100
8708/8708 [==============================] - 0s 9us/sample - loss: 7314.5107 - val_loss: 8746.6379
Epoch 68/100
8708/8708 [==============================] - 0s 10us/sample - loss: 7107.5930 - val_loss: 12327.0557
Epoch 69/100
8708/8708 [==============================] - 0s 10us/sample - loss: 7335.0336 - val_loss: 7185.9397
Epoch 70/100
8708/8708 [==============================] - 0s 11us/sample - loss: 6799.4566 - val_loss: 8275.4514
Epoch 71/100
8708/8708 [==============================] - 0s 11us/sample - loss: 7247.0496 - val_loss: 9602.8890
Epoch 72/100
8708/8708 [==============================] - 0s 11us/sample - loss: 7687.2127 - val_loss: 17860.0850
Epoch 73/100
8708/8708 [==============================] - 0s 11us/sample - loss: 8494.3264 - val_loss: 7219.7790
Epoch 74/100
8708/8708 [==============================] - 0s 10us/sample - loss: 6732.0348 - va

loss ~ 6243  add callback for early stop and increase epoch

In [31]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=1, patience=10, verbose=1, mode='min')
model_cplx_cb = build_mode([128, 64], 17) # 1 hidden layer

model_cplx_cb.compile(optimizer=tf.keras.optimizers.Adam(lr=0.008), loss='mse')
model_cplx_cb.summary()
history = model_cplx_cb.fit(x_training, y_training, epochs=200, batch_size=128, validation_data=(x_validation, y_validation), callbacks=[early_stop])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 128)               2304      
_________________________________________________________________
dense_15 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 65        
Total params: 10,625
Trainable params: 10,625
Non-trainable params: 0
_________________________________________________________________
Train on 8708 samples, validate on 2178 samples
Epoch 1/200
8708/8708 [==============================] - 0s 27us/sample - loss: 27343.1242 - val_loss: 22080.6950
Epoch 2/200
8708/8708 [==============================] - 0s 13us/sample - loss: 20595.8019 - val_loss: 21249.8600
Epoch 3/200
8708/8708 [==============================] - 0s 11us/sample - loss: 18907.4518 - val_loss: 1

Epoch 66/200
8708/8708 [==============================] - 0s 11us/sample - loss: 8122.5048 - val_loss: 8766.6286
Epoch 67/200
8708/8708 [==============================] - 0s 13us/sample - loss: 7247.1707 - val_loss: 7590.7244
Epoch 68/200
8708/8708 [==============================] - 0s 13us/sample - loss: 7130.3325 - val_loss: 9449.4030
Epoch 69/200
8708/8708 [==============================] - 0s 13us/sample - loss: 8017.3597 - val_loss: 7693.6483
Epoch 70/200
8708/8708 [==============================] - 0s 12us/sample - loss: 7049.1710 - val_loss: 8852.8772
Epoch 71/200
8708/8708 [==============================] - 0s 12us/sample - loss: 7051.0328 - val_loss: 7790.0979
Epoch 72/200
8708/8708 [==============================] - 0s 12us/sample - loss: 7763.1047 - val_loss: 8087.1003
Epoch 73/200
8708/8708 [==============================] - 0s 12us/sample - loss: 6601.0831 - val_loss: 7066.7502
Epoch 74/200
8708/8708 [==============================] - 0s 10us/sample - loss: 6580.9384 - val

loss ~ 5868

let's save the model

In [ ]:
check_point = ModelCheckpoint('/tmp/kaggle-bike/models/model.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_loss', min_delta=1, patience=10, verbose=1, mode='min')
model_cplx_cb_sv = build_mode([128, 64], 17) # 1 hidden layer

model_cplx_cb_sv.compile(optimizer=tf.keras.optimizers.Adam(lr=0.008), loss='mse')
model_cplx_cb_sv.summary()
history = model_cplx_cb_sv.fit(x_scale_temp, y_training, epochs=100, batch_size=128, validation_data=(x_val_temp, y_validation), callbacks=[early_stop, check_point])

In [32]:
x_training

,holiday,workingday,weather,temp,humidity,windspeed,hour,weekday,month,0,1,2,3,0,1,2,3
8793,0,1,2,31.98,55,11.0014,18,2,8,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
5101,0,1,3,20.50,100,12.9980,15,1,12,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1161,0,0,1,14.76,66,8.9981,1,6,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6141,0,0,2,4.10,46,46.0022,3,6,2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1639,0,1,1,13.94,76,6.0032,5,3,4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6542,0,1,1,15.58,32,23.9994,20,4,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2625,0,1,1,24.60,83,0.0000,7,4,6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
6408,0,0,2,12.30,52,19.9995,6,6,3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2174,0,1,3,22.96,83,27.9993,12,1,5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


predict test

In [35]:
test_no_datetime = break_datetime(test_data)
test_one_hot = one_hot_transform(test_no_datetime)
test_one_hot = test_one_hot.drop(['atemp', 'season'], axis=1)
test_one_hot

,holiday,workingday,weather,temp,humidity,windspeed,hour,weekday,month,0,1,2,3,0,1,2,3
0,0,1,1,10.66,56,26.0027,0,3,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,1,1,10.66,56,0.0000,1,3,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,1,1,10.66,56,0.0000,2,3,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0,1,1,10.66,56,11.0014,3,3,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,1,1,10.66,56,11.0014,4,3,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6488,0,1,2,10.66,60,11.0014,19,0,12,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6489,0,1,2,10.66,60,11.0014,20,0,12,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6490,0,1,1,10.66,60,11.0014,21,0,12,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6491,0,1,1,10.66,56,8.9981,22,0,12,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [62]:
t = np.reshape(test_data['datetime'].copy().values, (-1, 1))
result = np.concatenate((t, test_count), axis=1)
result

array([['2011-01-20 00:00:00', 9],
       ['2011-01-20 01:00:00', 15],
       ['2011-01-20 02:00:00', 11],
       ...,
       ['2012-12-31 21:00:00', 239],
       ['2012-12-31 22:00:00', 180],
       ['2012-12-31 23:00:00', 164]], dtype=object)

In [63]:
pd.DataFrame(result).to_csv("results/test_results2.csv", header=['datetime', 'count'], index=False)